In [ ]:
import os, sys, glob
import psycopg2
import pandas as pd
import xarray as xr
import numpy as np
from pathlib import Path
from datetime import timedelta

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

import plotly.express as px

# adds the package path to the Python path to make sure all the local imports work fine 
if os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))) not in sys.path:
    sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))

# local imports 
from wp4.constants import POLLUTANTS, DATA_DIR_CAMS, DATA_DIR_MERA, DATA_DIR_ERA5, DATA_DIR_PLOTS, DB_HOST, DB_NAME, DB_USER, DB_PASS
from wp4.baseline.spatial import calculate_wind_speed_direction

## Connect to the database and load some fire events into a dataframe 

In [ ]:
conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASS, host=DB_HOST)
cur = conn.cursor()

query = """
    SELECT id, datetime, ST_X(geometry), ST_Y(geometry), source, location, reference, type, info, frp
    FROM public.all_fire_events
    WHERE "frp" > 50 AND "frp" IS NOT NULL AND reference = 'Aqua' OR reference = 'Terra'
"""

df_fire_events = pd.read_sql_query(query,con=conn).rename(columns = {'st_x':'longitude', 'st_y':'latitude'})

conn.close()

## Load datasets

In [ ]:
pollutant = 'PM10' 
name_pollutant = POLLUTANTS[pollutant]["FULL_NAME"]
ds_era5 = xr.open_dataset(Path(DATA_DIR_ERA5).joinpath('era5.nc'))
ds_cams = xr.open_dataset(Path(DATA_DIR_CAMS).joinpath(f'{POLLUTANTS[pollutant]["CAMS"]}.nc'))

## Streamplot over CAMS data in matplotlib

A streamplot draws the streamlines of the vector flow, in this case representing the flow of wind. 

In [ ]:
for ind, row in df_fire_events.tail(1).iterrows():
    
    # select data for the time of the fire event from the datasets 
    
    ds_meteo = ds_era5.sel(
            time=row['datetime'].replace(minute=0), method='nearest'
    )
    
    ds_cams_fe = ds_cams.sel(
            time=row['datetime'].replace(minute=0), method='nearest'
    )

    ds_meteo = calculate_wind_speed_direction(ds_meteo)

    # select the u/v data from the meteo dataset
    u_10 = ds_meteo['u10']
    v_10 = ds_meteo['v10']
    
    # set up a numpy meshgrid matching the coordinates in the meteo dataset
    Y, X = np.mgrid[
        np.round_(ds_meteo.latitude[-1].values, 2):np.round_(ds_meteo.latitude[0].values, 2):44j,
        np.round_(ds_meteo.longitude[0].values, 2):np.round_(ds_meteo.longitude[-1].values, 2):62j
    ]
    
    # Initiate matplotlib figure, set size
    fig, ax = plt.subplots(figsize=(20, 10))
    # set the projection and add coastlines to the plot
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines()
    
    # first plot the CAMS data
    ds_cams_fe[POLLUTANTS["PM10"]["CAMS"]].plot(
        transform=ccrs.PlateCarree(),
        robust=True,
        cmap='coolwarm',
        extend='neither',
        facecolor="gray",
        cbar_kwargs={
            'shrink': 0.95,
            'label':f"{name_pollutant} Concentration (µg $m^{{-3}}$)",

        }
        )
    
    # set title
    ax.set_title(label=f'Streamplot for {row["datetime"].strftime("%B %e, %Y at %H:00")}')
    
    # Array containing the location of the fire event, we can use this as the start point in the stream plot.
    start_point = np.array(
            [
                [row['longitude']],
                [row['latitude']],
            ]
        )
    
    # create the streamline plot
    strm = ax.streamplot(
        X,
        Y,
        u_10.values,
        v_10.values,
        linewidth = 2,
        arrowsize=1.75,
        density=3,
        color='white',
        transform=ccrs.PlateCarree(),
        # uncomment the next line only show the stream passing through the location of the fire event
        # start_points = start_point.T, 
        
    )
    
    # add a point to the plot at the location of the fire event
    ax.scatter(
        row['longitude'],
        row['latitude'],
        marker='*',
        c='#ffba08',
        label='fire event',
        zorder=3,
        edgecolor='black',
        linewidth=1.5,
        s=500,
    )
    
    # add legend and display the plot
    ax.legend(loc='lower left', fontsize=12)
    plt.show()

## Quiver plot over CAMS data in Matplotlib

A quiverplot shows arrows based on the directional components U and V for the coordinates specified by X and Y.

In [ ]:
for ind, row in df_fire_events.tail(1).iterrows():
    
    # same idea as the previous plot, only showing a quiver plot now instead of a streamplot. 
    
    ds_meteo = ds_era5.sel(
            time=row['datetime'].replace(minute=0), method='nearest'
    )
    
    ds_cams_fe = ds_cams.sel(
            time=row['datetime'].replace(minute=0), method='nearest'
    )

    ds_meteo = calculate_wind_speed_direction(ds_meteo)

    u_10 = ds_meteo['u10']
    v_10 = ds_meteo['v10']
    
    Y, X = np.mgrid[
        np.round_(ds_meteo.latitude[-1].values, 2):np.round_(ds_meteo.latitude[0].values, 2):44j,
        np.round_(ds_meteo.longitude[0].values, 2):np.round_(ds_meteo.longitude[-1].values, 2):62j
    ]
    
    fig, ax = plt.subplots(figsize=(20, 10))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines()
    
    ds_cams_fe[POLLUTANTS["PM10"]["CAMS"]].plot(
        transform=ccrs.PlateCarree(),
        robust=True,
        cmap='coolwarm',
        extend='neither',
        facecolor="gray",
        cbar_kwargs={
            'shrink': 0.95,
            'label':f"{name_pollutant} Concentration (µg $m^{{-3}}$)",

        }
        )
    
    ax.set_title(label=f'Quiver plot for {row["datetime"].strftime("%B %e, %Y at %H:00")}')
    
    q = ax.quiver(X, Y, u_10.values, v_10.values, transform=ccrs.PlateCarree())
    ax.quiverkey(q, X=0.3, Y=1.1, U=10,
             label='Quiver key, length = 10', labelpos='E')

    
    ax.scatter(
        row['longitude'],
        row['latitude'],
        marker='*',
        c='#ffba08',
        label='fire event',
        zorder=3,
        edgecolor='black',
        linewidth=1.5,
        s=250,
    )
    
    ax.legend(loc='lower left', fontsize=12)
    plt.show()

## Interactive quiver plot over CAMS concentration raster

To run this you need to install the hvplot and geoviews packages

In [ ]:
import hvplot.xarray
import hvplot.pandas

row = df_fire_events[1:2]
    
ds_meteo = ds_era5.sel(
        time=row['datetime'].iloc[0].replace(minute=0), method='nearest'
)
    
ds_cams_fe = ds_cams.sel(
        time=row['datetime'].iloc[0].replace(minute=0), method='nearest'
)

ds_meteo = calculate_wind_speed_direction(ds_meteo)
    
cams_vis = ds_cams_fe.squeeze()[POLLUTANTS[pollutant]["CAMS"]].hvplot(
    projection=ccrs.PlateCarree(),
    coastline='10m',
)

cams_quiver = cams_vis * ds_meteo.hvplot.vectorfield(
    x='longitude',
    y='latitude',
    angle='wind_direction',
    mag='wind_speed',
    hover=False,
    geo=True).opts(magnitude='wind_speed')

# df = pd.DataFrame(row).T

points = row[['longitude', 'latitude']].hvplot.points(
    ('longitude', 'latitude'),
    geo=True,
    projection=ccrs.PlateCarree(),
    label='fire event'
 ).opts(color='red', size=10)

final_plot = cams_quiver * points
final_plot

## Windrose in plotly

In [ ]:
def create_windrose(df, title=None):
    """
    Function to create windrose from wind speed and direction data.
    Most of the code was taken from the following community post:
    
    https://community.plotly.com/t/wind-rose-with-wind-speed-m-s-and-direction-deg-data-columns-need-help/33274/3
    """
    
    #creating bins for magnitudes and directions 

    bins_mag= [0, 0.5, 1.5, 3.0, 5.0, 8, 11, 14, 20, 100]
    bins_mag_labels = ['0-0.5 m/s', '0.5-1.5 m/s',
                       '1.5-3 m/s', '3-5 m/s', '5-8 m/s',
                       '8-11 m/s', '11-14 m/s', '14-20 m/s', '20+ m/s']
    
    bins_dir = [0, 11.25, 33.75, 56.25, 78.75,101.25,123.75,146.25,168.75,191.25,213.75,236.25,258.75,281.25,303.75,326.25,348.75, 360.00]
    bins_dir_labels = ['N','NNE','NE','ENE','E','ESE','SE','SSE','S','SSW','SW','WSW','W','WNW','NW','NNW','North']
    
    # bin magnitude and direction from the meteo dataframe
    df['mag_binned'] = pd.cut(df['wind_speed'], bins_mag, labels=bins_mag_labels)
    df['dir_binned'] = pd.cut(df['wind_direction'],bins_dir, labels=bins_dir_labels)
    
    
    dfe = df[['mag_binned', 'dir_binned','u10']].copy() 
    dfe.rename(columns={'u10': 'freq'}, inplace=True)  #changing the last column to represent frequencies
    g = dfe.groupby(['mag_binned','dir_binned']).count() #grouping
    g.reset_index(inplace=True) 
    g['percentage'] = g['freq']/g['freq'].sum()
    g['percentage%'] = g['percentage']*100
    g['percentage%'] = g['percentage%'].round(2)
    g['Magnitude [m/s]'] = g['mag_binned']
    g = g.replace(r'North', 'N', regex=True) #replacing remaining Norths with N 
    
    
    # Create the plotly figure from with the data
    
    fig = px.bar_polar(
        g,
        r="percentage%",
        theta="dir_binned",
        color="Magnitude [m/s]",
        template="plotly_dark",
        color_discrete_sequence= px.colors.sequential.Plasma_r
    )
    
    fig.update_layout(polar_radialaxis_ticksuffix='%',title=title)
    
    return fig

In [ ]:
for ind, row in df_fire_events.head(1).iterrows():
    
    ds_meteo = ds_era5.sel(
        time=slice(
            row['datetime'].replace(minute=0) - timedelta(hours=0),
            row['datetime'].replace(minute=0) + timedelta(hours=1)
        )
    )
    
    ds_meteo = calculate_wind_speed_direction(ds_meteo, convention='from')
    
    ds_meteo_loc = ds_meteo.sel(
        latitude=row['latitude'],
        longitude=row['longitude'],
        method='nearest'
    )

    df = ds_meteo_loc.to_dataframe()[['wind_direction', 'wind_speed', 'u10']]
    fig = create_windrose(df, title='Wind info at the location of the fire event during the time of the fire event')
    fig.show()
    
    df = ds_meteo.to_dataframe()[['wind_direction', 'wind_speed', 'u10']]
    fig = create_windrose(df, title='Wind info for Ireland during the time of the fire event')
    fig.show()